In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle


In [3]:
data=pd.read_csv('top10K-TMDB-movies.csv')
df=pd.DataFrame(data)
df

,id,title,genre,original_language,overview,popularity,release_date,vote_average,vote_count
0,278,The Shawshank Redemption,"Drama,Crime",en,Framed in the 1940s for the double murder of h...,94.075,1994-09-23,8.7,21862
1,19404,Dilwale Dulhania Le Jayenge,"Comedy,Drama,Romance",hi,"Raj is a rich, carefree, happy-go-lucky second...",25.408,1995-10-19,8.7,3731
2,238,The Godfather,"Drama,Crime",en,"Spanning the years 1945 to 1955, a chronicle o...",90.585,1972-03-14,8.7,16280
3,424,Schindler's List,"Drama,History,War",en,The true story of how businessman Oskar Schind...,44.761,1993-12-15,8.6,12959
4,240,The Godfather: Part II,"Drama,Crime",en,In the continuing saga of the Corleone crime f...,57.749,1974-12-20,8.6,9811
...,...,...,...,...,...,...,...,...,...
9995,10196,The Last Airbender,"Action,Adventure,Fantasy",en,"The story follows the adventures of Aang, a yo...",98.322,2010-06-30,4.7,3347
9996,331446,Sharknado 3: Oh Hell No!,"Action,TV Movie,Science Fiction,Comedy,Adventure",en,The sharks take bite out of the East Coast whe...,12.490,2015-07-22,4.7,417
9997,13995,Captain America,"Action,Science Fiction,War",en,"During World War II, a brave, patriotic Americ...",18.333,1990-12-14,4.6,332
9998,2312,In the Name of the King: A Dungeon Siege Tale,"Adventure,Fantasy,Action,Drama",en,A man named Farmer sets out to rescue his kidn...,15.159,2007-11-29,4.7,668


In [4]:
df.isna().sum()

id                    0
title                 0
genre                 3
original_language     0
overview             13
popularity            0
release_date          0
vote_average          0
vote_count            0
dtype: int64

In [5]:
null_rows = df[df['genre'].isnull()]
print(null_rows)

         id                    title genre original_language  \
3361  50472     Anplagghed al cinema   NaN                it   
7821  43211        7 Kilos in 7 Days   NaN                it   
8518  57114  Amore, bugie e calcetto   NaN                en   

                                               overview  popularity  \
3361  A queue at the ATM machine, a displaced family...       4.420   
7821  Two not very clever young doctors open a fitne...       5.885   
8518                                                NaN       4.709   

     release_date  vote_average  vote_count  
3361   2006-11-26           7.0         313  
7821   1986-02-02           6.0         212  
8518   2008-04-04           5.8         200  


In [6]:
df['genre'] = df['genre'].fillna('Action,Drama	')

In [7]:
df['genre'] .isna().sum()

0

In [8]:
null_rows = df[df['overview'].isnull()]
print(null_rows)

          id                                      title           genre  \
4150   38537                               Nati stanchi          Comedy   
6973   31359                      Would I Lie to You? 2          Comedy   
7941    2029                                     Tanguy          Comedy   
8518   57114                    Amore, bugie e calcetto  Action,Drama\t   
9293   17413                                  Incognito          Comedy   
9440  516043                            Arrivano i prof          Comedy   
9620  154512                           Lightning Strike          Comedy   
9792   42426  A spasso nel tempo - L'avventura continua  Comedy,Fantasy   
9825   77000                                  Bar Sport   Comedy,Family   
9889   42436                         Natale in crociera          Comedy   
9915   57216                          A Natale mi sposo          Comedy   
9922  408238           The New Adventures of Cinderella          Comedy   
9989  244514             

In [9]:
df['overview'] = df['overview'].fillna('An uproarious comedy-drama unfolds as a tight-knit community navigates love, laughter, and unexpected twists in Harmonyville')

In [10]:
df['overview'] .isna().sum()

0

In [11]:
df.columns

Index(['id', 'title', 'genre', 'original_language', 'overview', 'popularity',
       'release_date', 'vote_average', 'vote_count'],
      dtype='object')

In [12]:
movies=df[['id','title','genre','overview','release_date']]

In [13]:
movies['release_date'] = movies['release_date'].str[:4]

C:\Users\akhil\AppData\Local\Temp\ipykernel_9144\2571805512.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['release_date'] = movies['release_date'].str[:4]


In [14]:
movies

,id,title,genre,overview,release_date
0,278,The Shawshank Redemption,"Drama,Crime",Framed in the 1940s for the double murder of h...,1994
1,19404,Dilwale Dulhania Le Jayenge,"Comedy,Drama,Romance","Raj is a rich, carefree, happy-go-lucky second...",1995
2,238,The Godfather,"Drama,Crime","Spanning the years 1945 to 1955, a chronicle o...",1972
3,424,Schindler's List,"Drama,History,War",The true story of how businessman Oskar Schind...,1993
4,240,The Godfather: Part II,"Drama,Crime",In the continuing saga of the Corleone crime f...,1974
...,...,...,...,...,...
9995,10196,The Last Airbender,"Action,Adventure,Fantasy","The story follows the adventures of Aang, a yo...",2010
9996,331446,Sharknado 3: Oh Hell No!,"Action,TV Movie,Science Fiction,Comedy,Adventure",The sharks take bite out of the East Coast whe...,2015
9997,13995,Captain America,"Action,Science Fiction,War","During World War II, a brave, patriotic Americ...",1990
9998,2312,In the Name of the King: A Dungeon Siege Tale,"Adventure,Fantasy,Action,Drama",A man named Farmer sets out to rescue his kidn...,2007


In [15]:
movies['tags'] = movies['genre'] + ' ' + movies['overview']

C:\Users\akhil\AppData\Local\Temp\ipykernel_9144\1618478180.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['tags'] = movies['genre'] + ' ' + movies['overview']


In [16]:
movies

,id,title,genre,overview,release_date,tags
0,278,The Shawshank Redemption,"Drama,Crime",Framed in the 1940s for the double murder of h...,1994,"Drama,Crime Framed in the 1940s for the double..."
1,19404,Dilwale Dulhania Le Jayenge,"Comedy,Drama,Romance","Raj is a rich, carefree, happy-go-lucky second...",1995,"Comedy,Drama,Romance Raj is a rich, carefree, ..."
2,238,The Godfather,"Drama,Crime","Spanning the years 1945 to 1955, a chronicle o...",1972,"Drama,Crime Spanning the years 1945 to 1955, a..."
3,424,Schindler's List,"Drama,History,War",The true story of how businessman Oskar Schind...,1993,"Drama,History,War The true story of how busine..."
4,240,The Godfather: Part II,"Drama,Crime",In the continuing saga of the Corleone crime f...,1974,"Drama,Crime In the continuing saga of the Corl..."
...,...,...,...,...,...,...
9995,10196,The Last Airbender,"Action,Adventure,Fantasy","The story follows the adventures of Aang, a yo...",2010,"Action,Adventure,Fantasy The story follows the..."
9996,331446,Sharknado 3: Oh Hell No!,"Action,TV Movie,Science Fiction,Comedy,Adventure",The sharks take bite out of the East Coast whe...,2015,"Action,TV Movie,Science Fiction,Comedy,Adventu..."
9997,13995,Captain America,"Action,Science Fiction,War","During World War II, a brave, patriotic Americ...",1990,"Action,Science Fiction,War During World War II..."
9998,2312,In the Name of the King: A Dungeon Siege Tale,"Adventure,Fantasy,Action,Drama",A man named Farmer sets out to rescue his kidn...,2007,"Adventure,Fantasy,Action,Drama A man named Far..."


In [17]:
movies=movies.drop(columns=['overview'])

In [18]:
movies

,id,title,genre,release_date,tags
0,278,The Shawshank Redemption,"Drama,Crime",1994,"Drama,Crime Framed in the 1940s for the double..."
1,19404,Dilwale Dulhania Le Jayenge,"Comedy,Drama,Romance",1995,"Comedy,Drama,Romance Raj is a rich, carefree, ..."
2,238,The Godfather,"Drama,Crime",1972,"Drama,Crime Spanning the years 1945 to 1955, a..."
3,424,Schindler's List,"Drama,History,War",1993,"Drama,History,War The true story of how busine..."
4,240,The Godfather: Part II,"Drama,Crime",1974,"Drama,Crime In the continuing saga of the Corl..."
...,...,...,...,...,...
9995,10196,The Last Airbender,"Action,Adventure,Fantasy",2010,"Action,Adventure,Fantasy The story follows the..."
9996,331446,Sharknado 3: Oh Hell No!,"Action,TV Movie,Science Fiction,Comedy,Adventure",2015,"Action,TV Movie,Science Fiction,Comedy,Adventu..."
9997,13995,Captain America,"Action,Science Fiction,War",1990,"Action,Science Fiction,War During World War II..."
9998,2312,In the Name of the King: A Dungeon Siege Tale,"Adventure,Fantasy,Action,Drama",2007,"Adventure,Fantasy,Action,Drama A man named Far..."


In [19]:
movies.shape

(10000, 5)

In [20]:
movies['tags'] = movies['tags'].str.lower().replace('[.,'':!-]', ' ', regex=True)

In [21]:
movies['title'] = movies['title'].replace('[.,'':!-]', ' ', regex=True)

In [22]:
movies['genre'] = movies['genre'].str.lower().replace('[.,'':!-]', ' ', regex=True)

In [23]:
movies

,id,title,genre,release_date,tags
0,278,The Shawshank Redemption,drama crime,1994,drama crime framed in the 1940s for the double...
1,19404,Dilwale Dulhania Le Jayenge,comedy drama romance,1995,comedy drama romance raj is a rich carefree ...
2,238,The Godfather,drama crime,1972,drama crime spanning the years 1945 to 1955 a...
3,424,Schindler's List,drama history war,1993,drama history war the true story of how busine...
4,240,The Godfather Part II,drama crime,1974,drama crime in the continuing saga of the corl...
...,...,...,...,...,...
9995,10196,The Last Airbender,action adventure fantasy,2010,action adventure fantasy the story follows the...
9996,331446,Sharknado 3 Oh Hell No,action tv movie science fiction comedy adventure,2015,action tv movie science fiction comedy adventu...
9997,13995,Captain America,action science fiction war,1990,action science fiction war during world war ii...
9998,2312,In the Name of the King A Dungeon Siege Tale,adventure fantasy action drama,2007,adventure fantasy action drama a man named far...


In [24]:
vectorizer= TfidfVectorizer(max_features=10000,)

In [25]:
vector=vectorizer.fit_transform(movies['tags'])

In [26]:
vector.shape

(10000, 10000)

In [27]:
similarity = cosine_similarity(vector)

In [28]:
def recommend_by_title(movie_title, top_n=10):
    movie_title=movie_title.lower()
    index = movies[movies['title'].str.lower()== movie_title].index[0]
    distance = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda vector: vector[1])
    for i in distance[1:top_n+1]:  
        print(movies.iloc[i[0]].title)

In [66]:
recommend_by_title("Spider Man")

Spider Man  Into the Spider Verse
Spider Man 3
Arachnophobia
Spider Man  Homecoming
Spider Man 2
Spider Man  No Way Home
The Amazing Spider Man
The Amazing Spider Man 2
Minnal Murali
Jurassic Hunt


In [67]:
def recommend_by_year(release_year, top_n=10):
    movies_in_year = movies[movies['release_date'] == str(release_year)].head(top_n)

    if not movies_in_year.empty:
        for index, row in movies_in_year.iterrows():
            print(row['title'])
    else:
        print(f"No movies found for the year {release_year}")


In [68]:
recommend_by_year(1990)

GoodFellas
Dances with Wolves
Awakenings
Misery
Edward Scissorhands
Dreams
Miller's Crossing
A Grand Day Out
Dragon Ball Z  Bardock   The Father of Goku
Back to the Future Part III


In [69]:
def recommend_by_genre(input_genre, top_n=10):
    input_genre = input_genre.lower()
    
    movies_with_genre = movies[movies['tags'].str.contains(input_genre)].head(top_n)
    if not movies_with_genre.empty:
        for index, row in movies_with_genre.iterrows():
            print(row['title'])
    else:
        print(f"No movies found for the genre {input_genre}")


In [70]:
recommend_by_genre("romance")

Dilwale Dulhania Le Jayenge
Your Eyes Tell
Dou kyu sei – Classmates
Your Name 
Gabriel's Inferno
Gabriel's Inferno  Part II
Gabriel's Inferno  Part III
Forrest Gump
Cinema Paradiso
Violet Evergarden  The Movie


In [71]:
def recommend_movies(input_value, top_n=10):
    
    if str(input_value).lower() in movies['genre'].str.lower().unique():
       
        recommend_by_genre(str(input_value), top_n)
    else:
        try:
            
            year = int(input_value)
            recommend_by_year(year, top_n)
        except ValueError:
           
            recommend_by_title(str(input_value), top_n)



In [72]:
recommend_movies(1990)

GoodFellas
Dances with Wolves
Awakenings
Misery
Edward Scissorhands
Dreams
Miller's Crossing
A Grand Day Out
Dragon Ball Z  Bardock   The Father of Goku
Back to the Future Part III


In [73]:
recommend_movies("spider man")

Spider Man  Into the Spider Verse
Spider Man 3
Arachnophobia
Spider Man  Homecoming
Spider Man 2
Spider Man  No Way Home
The Amazing Spider Man
The Amazing Spider Man 2
Minnal Murali
Jurassic Hunt


In [74]:
recommend_movies("drama crime")

The Shawshank Redemption
The Godfather
The Godfather  Part II
The Green Mile
GoodFellas
Once Upon a Time in America
City of God
The Usual Suspects
Bad Genius
Trainspotting


In [29]:
pickle.dump(movies, open('movies_list.pkl', 'wb'))

In [30]:
pickle.dump(similarity, open('similarity.pkl', 'wb'))